# Notebook to Visualize pcd,rgb,point pridictions 

The purpose of the notebook is to visualize the pcd rgb and posenet corresponding to model

In [38]:
import pandas as pd
import cufflinks
cufflinks.go_offline(connected=True)
from ipywidgets import interact_manual
import scipy

from visualizeutils import (selectData, imageshow, pcdrgb)


In [39]:
artifacts = pd.read_csv('../posenetVSheightmodel.csv')
artifacts.shape

(910, 23)

In [40]:
artifacts.head()

,pcd_file,rgb_file,age,model_id,error,score,json_value_pose_1_nose_score,json_value_pose_1_leftAnkle_score,json_value_pose_1_leftEar_score,json_value_pose_1_leftElbow_score,...,json_value_pose_1_leftShoulder_score,json_value_pose_1_leftWrist_score,json_value_pose_1_rightAnkle_score,json_value_pose_1_rightEar_score,json_value_pose_1_rightElbow_score,json_value_pose_1_rightEye_score,json_value_pose_1_rightHip_score,json_value_pose_1_rightKnee_score,json_value_pose_1_rightShoulder_score,json_value_pose_1_rightWrist_score
0,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,650,20190708-0919_2379-595height,7.760607,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,650,20190806-1551_220-55height,5.824747,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,650,GAPNet_height_s1,8.708862,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,650,q3-cnndepthmap-resnet-height_run_13,11.006977,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,/mnt/cgminbmzqa/qrcode/IN_AAH_RJ_00355/measure...,650,q3-depthmap-plaincnn-height-100-95k_run_03,9.864527,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Data cleaning

 Remove the age at 0 and age above 5 years(i.e 1825 days)

In [41]:
artifacts = artifacts[artifacts['age'] <= 1825]
artifacts = artifacts[artifacts['age'] > 0]
artifacts = artifacts.fillna(0)

Plot count of artifacts with age


In [42]:
artifacts['age'].iplot(kind='hist',
                       xTitle='Age',
                       yTitle='count',
                       title='Age Distribution')

#####  No of unique model

In [43]:
model_id = artifacts['model_id'].unique().tolist()
model_id

['20190708-0919_2379-595height',
 '20190806-1551_220-55height',
 'GAPNet_height_s1',
 'q3-cnndepthmap-resnet-height_run_13',
 'q3-depthmap-plaincnn-height-100-95k_run_03',
 'q3-depthmap-plaincnn-height-100-95k_run_04',
 'q3-depthmap-plaincnn-height-95k_run_03',
 'q3-depthmap-plaincnn-height-95k_run_04',
 'q3_depthmap_height_run_01',
 'q3_depthmap_height_run_02']

#### Interatcive tool to play and find any corelation 

In [44]:
@interact_manual
def show_articles_more_than(age_start=(0, 1825, 180),
                            model_id=model_id,
                            x=list(artifacts.select_dtypes('number')
                                   .columns[2:],),
                            y=list(artifacts.select_dtypes('number')
                                   .columns)[1:2],
                            error=[0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.5, 2,
                                   2.5, 3.0, 3.5, 4.0, 10.0, 20.0, 40, 60]):
    dff = artifacts.loc[artifacts['model_id'] == model_id]
    dff = dff.loc[dff['age'] > age_start]
    dff = dff.loc[dff['error'] < error]
    xaxis = dff[x].tolist()
    yaxis = dff[y].tolist()
    print(len(xaxis))
    print("Cofficient constant R", scipy.stats.pearsonr(xaxis, yaxis)[0])
    dff.iplot(kind='scatter', x=x, y=y, mode='markers')

interactive(children=(IntSlider(value=900, description='age_start', max=1825, step=180), Dropdown(description=…

### Visualizing the pcd,rgb,posenet with gapnet model 


select data from gapnet model where model_id[2] is gapnet

In [45]:
df = selectData(artifacts, model_id[2])
df.shape

(98, 24)

show the image and save that in corresponding file

In [59]:
#commenting this line as we can't show image with face
imageshow(df,
          result_grid_filename='gridgapnet.jpg',
          result_figsize_resolution=80)


"imageshow(df,\n          result_grid_filename='gridgapnet.jpg',\n          result_figsize_resolution=80)\n"

pcd and rgb of particular index

In [47]:
#commenting this line as we can't show image with face
pcdrgb(df, 1)

### Visualizing the pcd,rgb,posenet with pointnet model 


In [48]:
df = selectData(artifacts, model_id[1])
df.shape

(114, 24)

In [49]:
imageshow(df,
          result_grid_filename='gridpointnet.jpg',
          result_figsize_resolution=80)

In [50]:
pcdrgb(df, 0)

### Visualizing the pcd,rgb,posenet with deptmap model(q3-cnndepthmap-resnet-height_run_13)

In [51]:
df = selectData(artifacts, model_id[3])
df.shape

(80, 24)

In [54]:
imageshow(df,
          result_grid_filename='griddepthmap.jpg',
          result_figsize_resolution=80)

In [55]:
pcdrgb(df, 1)

Findings
1. Disturbance of the pcd clearly give idea about error
2. Child with a big face in image-->high error
3. Sudden chage in background color -->high error
4. As black dosent relfect light some distoration in pcd at hair
5. Posenet fails most of the time in detecting error
6. Blurry image-->high error

TODO
1. Visualize light score
2. Retrain of gapnet will be very helpful
